## CHAPTER 7 앙상블 학습과 랜덤포레스트
- 결정 트리의 앙상블을 랜덤포레스트라 하며 오늘날 가장 강력한 머신러닝 모델 중 하나이다.
- 프로젝트의 마지막에 이미 만든 여러 괜찮은 예측기를 연결하여 더 좋은 예측기를 만든다.
![test](./img/앙상블1.png)
- 더 좋은 분류기를 만드는 매우 간단한 방법은 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것이다. (즉, 여러 예측기를 통해 다수결 투표를 하여 가장 많이 선택된 클래스를 채택한다.)
![test](./img/다수결투표.png)
- 이 다수결 투표 분류기가 앙상블에 포함된 개별 분류기 중 가장 뛰어난 것보다도 높은 경우가 많다.
- 앙상블 방법은 예측기가 가능한 서로 독립적일 때 최고의 성능을 발휘 한다.
> - 같은 데이터로 훈련시키기 때문에 같은 종류의 오차를 만들기 쉽다. 따라서 잘못된 클래스가 다수인 경우가 많고 앙상블의 정확도가 낮아진다.
- 다양한 분류기를 얻는 한 가지 방법은 각기 다른 알고리즘으로 학습시키는 것이다. 
> - 이렇게 하면 매우 다른 종류의 오차를 만들 가능성이 높기 때문에 앙상블 모델의 정확도를 향상

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42) # moon data_set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [3]:
log_clf = LogisticRegression() # LogisticRegression
rnd_clf = RandomForestClassifier() # RandomForestClassifier
svm_clf = SVC() # SVM

In [4]:
voting_clf = VotingClassifier(estimators=[('lr',log_clf),
                                        ('rf',rnd_clf),
                                        ('svc',svm_clf),
                                        ],voting='hard') # model create
voting_clf.fit(X_train,y_train) # model training

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [5]:
from sklearn.metrics import accuracy_score

In [6]:
for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
    clf.fit(X_train,y_train) 
    y_pred = clf.predict(X_test) # Performance check
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred)) # score check

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.888
VotingClassifier 0.888


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


- 예상대로 투표기반 분류기가 다른 개별 분류기보다 성능이 조금 더 높다.
---------
- 모든 분류기가 클래스의 확률을 예측할 수 있으면 (즉, predict_proba() 메서드가 있다면), 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있다. 이를 __간접 투표__라 한다.
- 이 방식은 확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식 보다 성능이 높다.
> - voting = "soft"로 바꾸고 모든 분류기가 클래스의 확률을 추정할 수 있으면 된다.
> - SVC는 클래스의 확률을 제공하지 않으므로 probability 매개변수를 True로 지정해야 한다.

In [7]:
log_clf = LogisticRegression() # LogisticRegression
rnd_clf = RandomForestClassifier() # RandomForestClassifier
svm_clf = SVC(probability=True) # SVM

voting_clf = VotingClassifier(estimators=[('lr',log_clf),
                                        ('rf',rnd_clf),
                                        ('svc',svm_clf),
                                        ],voting='soft') # soft voting
voting_clf.fit(X_train,y_train) # model training

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [8]:
from sklearn.metrics import accuracy_score

for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
    clf.fit(X_train,y_train) 
    y_pred = clf.predict(X_test) # Performance check
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred)) # score check

LogisticRegression 0.864
RandomForestClassifier 0.872
SVC 0.888
VotingClassifier 0.904


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


- 91%의 정확도를 얻음을 확인

## 배깅과 페이스팅
- 같은 알고리즘을 사용하지만 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 방법이 있다.
> - ~~직접투표방식과 간접투표방식은 각기 다른 훈련 알고리즘을 사용하였다.~~
- 훈련 세트에서 중복을 허용하여 샘플링하는 방식을 __배깅(bootstrap aggregating)__
- 중복을 허용하지 않고 샘플링하는 방식을 __페이스팅(pasting)__
> - 다시말해 배깅과 페이스팅에서는 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용할 수 있다.
> - 배깅만 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링 할 수 있다.

![test](./img/배깅.png)

- 모든 예측기가 훈련을 마지면 앙상블은 모든 예칙기의 예측을 모아서 새로운 샘플에 대한 예측을 만든다.
- 수집함수는 전형적으로 분류일 때는 통계적 최빈값 (즉, 직접 투표 분류기처럼 가장 많은 예측 결과)이고 회귀에 대해서는 평균을 계산한다.
- 일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬때와 비교해 비슷하지만 분산은 줄어든다. (분산이 줄어든다는 뜻은 그 만큼 과대적합에 위험에서 벗어난다는 뜻)

In [9]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [10]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(),n_estimators=500,
                           max_samples=100,bootstrap=True,n_jobs = -1)
bag_clf.fit(X_train,y_train)
y_pred = bag_clf.predict(X_test)

- n_estimators = 500 : 500개의 트리 생성
- max_samples = 100 : 한개의 트리당 100개의 샘플을 뽑는다.
- bootstrap = True : 중복을 허용한 리샘플링
- n_jobs = -1 : 가용가능한 모든 CPU 코어 수를 지정

In [11]:
from sklearn.metrics import accuracy_score

In [12]:
print(accuracy_score(y_test,y_pred)) # 0.928

0.92


![test](./img/배깅결정트리.png)

- 앙상블의 예측이 결정 트리 하나의 예측보다 더 일반화가 잘 되었음을 알 수 있다.
- 앙상블은 비슷한 평향에서 더 작은 분산을 만든다.
> - 훈련세트의 오차 수가 거의 비슷하지만 결정 경계는 덜 불규칙하다.
- 부스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로 __배깅이 페이스팅보다 편향이 조금 높다.__ 하지만 이는 예측기들의 __상관관계를 줄이므로 앙상블의 분산을 감소__시킨다. 전반적으로 배깅이 더 나은 모델을 만들기 때문에 일반적으로 더 선호한다.

### oob평가
- 배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링 되고 어떤 것은 전혀 선택되지 않을 수 있다. BaggingClassifer는 기본값으로 중복을 허용하여 (bootstrap=True) 훈련 세트의 크기 만큼인 m개의 샘플을 선택한다.
- 이는 평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링된다는 것을 의미
- 선택되지 않은 훈련 샘플의 나머지 37%를 oob(out of bag)샘플이라 부른다
> - 예측기마다 남겨진 37%는 모두 다르다.
- 예측기가 훈련되는 동안에는 oob 샘플을 사용하지 않으므로 검증 세트나 교차 검증을 사용하지 않고 oob 샘플을 사용해 평가할 수 있다. 
> - __앙상블의 평가는 각 예측기의 oob평가를 평균하여 얻는다.__

In [13]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(),n_estimators = 500,
                           bootstrap = True, n_jobs = -1, oob_score = True)
bag_clf.fit(X_train,y_train)
bag_clf.oob_score_

0.8933333333333333

- oob_score = True로 지정하면 훈련이 끝난 후 자동으로 oob평가를 수행
- 즉, model을 훈련 시키는 동안 oob샘플은 훈련 대상에서 제외됨으로 oob샘플을 사용해 모델을 평가 할 수 있다.

In [14]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test,y_pred) # oob 샘플 점수와 비슷함을 알 수 있다.

0.912

In [15]:
bag_clf.oob_decision_function_ # oob샘플에 대한 결정 함수 확인 [음성/양성]

array([[0.3248731 , 0.6751269 ],
       [0.34871795, 0.65128205],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.07954545, 0.92045455],
       [0.36931818, 0.63068182],
       [0.01470588, 0.98529412],
       [0.98837209, 0.01162791],
       [0.9895288 , 0.0104712 ],
       [0.79081633, 0.20918367],
       [0.01970443, 0.98029557],
       [0.79329609, 0.20670391],
       [0.85416667, 0.14583333],
       [0.96      , 0.04      ],
       [0.05263158, 0.94736842],
       [0.        , 1.        ],
       [0.97790055, 0.02209945],
       [0.95555556, 0.04444444],
       [0.99456522, 0.00543478],
       [0.        , 1.        ],
       [0.37288136, 0.62711864],
       [0.90909091, 0.09090909],
       [1.        , 0.        ],
       [0.96685083, 0.03314917],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.66857143, 0.33142857],
       [0.

## 램덤 패치와 랜덤 서브스페이스
- BaggingClassifier는 특정 샘플링도 지원한다.
> - max_features: 최대 특성 선택
> - bootstrap_feature: 랜덤 특성 선택
- 따라서 각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련된다.
> 특히 이미지와 같은 매우 고차원의 데이터셋을 다룰 때 유용하다.
- 훈련 특성과 샘플을 모두 샘플링 하는 것을 랜덤 패치 방식이라고 하고 훈련 샘플을 모두 사용(max_samples = 1.0)하고 특성은 샘플링하는 것을 랜덤 서브스페이스 방식이라 한다.


## 랜덤 포레스트
- 랜덤 포레스트는 배깅 방법을 적용한 결정트리의 앙상블이다. 
> - 전형적으로 max_samples를 훈련 세트의 크기로 지정
> - Bagging에 Decision을 넣어 만드는 방법 대신 결정 트리에 최적하되어 사용하기 편리한 것이 랜덤포레스트 방법이다.
- 랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입한다.
> - 결국 트리를 더욱 다양하게 만들고 편향을 손해 보는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 만든다.

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=16,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

- n_estimatiors = 500은 500개의 tree 생성
- max_leaf_nodes = 16 최대 16개의 리프노드를 갖는다.
- n_jobs = -1 가용가는한 CPU 코어를 사용

In [18]:
y_pred_rf = rnd_clf.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred) # oob 샘플 점수와 비슷함을 알 수 있다.

0.912

## 특성의 중요도
- 랜덤 포레스트의 또 다른 장점은 특성의 상대적 중요도를 측정하기 쉽다.
> - 어떤 특성을 사용한 노드가 (랜덤 포레스트에 있는 모든 트리에 걸쳐서) 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도를 측정한다.
> - 더 정확히 말하면 가중치 평균이며 각 노드의 가중치는 연관된 훈련 샘플 수 와 같다.
>> - 노드에 사용된 각 특성 별로(현재 노드의 샘플 비율 X 불순도) - (왼쪽 자식 노드의 샘플 비율 X 불순도) - (오른쪽 자식 노드의 샘플 비율 X 불순도)
>> - 그 다음 특성 중요도 합이 1이 되도록 전체 합으로 나누어 정규화 한다.
>> - 여기서 샘플 비율은 트리 전체 샘플 수에 대한 비율이다.
>> - 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 모두 계산하여 더한 후 트리 수로 나눈 것이다.
> - 사이킷런은 훈련이 끝난 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결괏값을 정규화 한다.
>> - 이 값은 feature_importances_ 변수에 저장

In [20]:
from sklearn.datasets import load_iris

In [21]:
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500,n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [22]:
for name, score in zip (iris['feature_names'],rnd_clf.feature_importances_):
    print(name, score) # petal, length가 핸덤 포레스트 모델을 훈련 시킴에 있어서 중요한 특성이다.

sepal length (cm) 0.09430277497419999
sepal width (cm) 0.022392526346413304
petal length (cm) 0.4427560086400761
petal width (cm) 0.4405486900393108


## 부스팅
- 부스팅(boosting)은 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법을 말한다.
- 부스팅 방법의 아이디어는 앞의 모델을 보완해나가면서 일련의 예측기를 학습시키는 것이다.

## 아다부스트(AdaBoost)
- 이전 예측기를 보완하는 새로운 예측기를 만드는 방법은 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 것이다.
- 이렇게 하면 새로운 예측기는 학습하기 어려운 샘플에 점점 더 맞춰지게 된다.
![test](./img/아다부스트.png)
- 첫 번째 분류기 훈련세트에서 훈련을 시키고 예측을 만드는데 잘못 분류된 훈련 샘플의 가주치를 상대적으로 높여 다음 두 번째 분류기는 업데이트된 가중치를 사용해 훈련 세트에서 훈련하고 예측을 만들고 다시 가중치를 업데이트 하는 식으로 계속된다.
- 연속된 학습 기법에는 중요한 단점이 하나 있다. 각 예측기는 이전 예측기가 훈련되고 평가된 후에 학습 될 수 있기 때문에 병렬화를 할 수 없다. 결국 배깅이나 페이스팅 만큼 확정성이 높지 않다.
> - 사이킷런은 SAMME라는 아다부스트 다중 클래스 버전을 시용한다. 클래스가 두 개 뿐일 때는 SAMME가 아다부스트와 동일하다.
> - 예측기가 클래스의 확률을 추정할 수 있다면(즉, predict_proba()메서드가 있다면) 사이킷런은 SAMME.R이라는 SAMME의 변종을 사용한다.

In [23]:
from sklearn.ensemble import AdaBoostClassifier

In [24]:
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=200,
                            algorithm='SAMME.R',learning_rate=0.5)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.5, n_estimators=200, random_state=None)

In [25]:
y_pred_ada = ada_clf.predict(X_test)

In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_ada)

0.896

- 200개의 얕은 결정 트리를 기반으로 분류기를 훈련시킨다.
- max_depth = 1 은 결정 노드 하나와 리프 노드 두개로 이루어진 트리이다.

## 그래디언트 부스팅
- 인기가 높은 또 하나의 부스팅 알고리즘은 그래디언트 부스팅이다. 
- 아다부스트처럼 그래디언트 부스팅은 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가한다. 
> - 하지만 아다부스트처럼 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 잔여 오차(residual error)에 새로운 예측기를 학습시킨다.

In [43]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [44]:
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [45]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

- 첫 번째 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor을 훈련시킨다.

In [46]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

- 다음 두 번째 예측기가 만든 잔여 오차에 세 번째 회귀 모델을 훈련시킨다.

In [47]:
X_new = np.array([[0.8]])

In [48]:
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1,tree_reg2,tree_reg3))

In [49]:
y_pred

array([0.75026781])

- 이제 세 개의 트리를 포함하는 앙상블 모델이 생겼다.
- 새로운 샘플에 대한 예측을 만들려면 모든 트리의 예측을 더하면 된다.

![test](./img/그래디언트부스팅.png)

- 왼쪽 열은 이 세 트리의 예측이고 오른쪽 열은 앙상블의 예측이다.
> - 첫 번째 행에서는 앙상블에 트리가 하나만 있어서 첫 번째 트리의 예측과 완전히 같다.
> - 두 번째 행에서는 새로운 트리가 첫 번째 트리의 잔여 오차에 대해 학습되었다.
>> - 오른쪽의 앙상블 예측이 두 개의 트리 예측의 합과 같은 것을 볼 수 있다.
> - 비슷하게 세 번째 행에서는 또 다른 트리가 두 번째 트리의 잔여 오차에 훈련되었다.
> - 트리가 앙상블에 추가될수록 앙상블의 예측이 점차 좋아지는 것을 알 수 있다.


In [50]:
from sklearn.ensemble import GradientBoostingRegressor

In [51]:
gbrt = GradientBoostingRegressor(max_depth=2,n_estimators=3,learning_rate=1.0)

- max_depth: 트리의 최대 깊이
- n_estimators: 트리 수
- learning_rate: 각 트리의 기여 정도를 조절
> - 0.1처럼 낮게 설정하면 앙상블을 훈련 세트에 학습시키기 위해 많은 트리가 필요하지만 일반적으로 예측의 성능은 좋아진다. 이는 축소라고 부르는 규제 방법이다.

![test](./img/앙상블.png)

- 작은 학습률로 훈련 시킨 두 개의 앙상블이다.
> - 왼쪽은 훈련 세트를 학습하기에는 트리가 충분하지 않고 오른쪽은 트리가 너무 많아 훈련 세트에 과대적합되었다.
- 최적의 트리 수를 찾기 위해서는 조기 종료 기법(4장)을 사용할 수 있다. 
> - 간단하게 구현하려면 staged_predict() 메서드를 사용한다. 
>> - 이 메서드는 훈련의 각 단계 (트리 하나, 트리 두 개 등)에서 앙상블에 의해 만들어진 예측기를 순회하는 반복자(iterator)를 반환한다.

In [52]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [55]:
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [58]:
X_train,X_val,y_train,y_val = train_test_split(X,y)

In [59]:
gbrt = GradientBoostingRegressor(max_depth=2,n_estimators=120)
gbrt.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=120, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [60]:
errors = [mean_squared_error(y_val,y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

In [61]:
gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=67, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

![test](./img/튜닝.png)

- 많은 수의 트리를 먼저 훈련시키고 최적의 수를 찾기 위해 살펴보는 대신 실제로 훈련을 중지하는 방법으로 조기 종료를 구현할 수도 있다. warm_start = True를 설정하면 사이킷런이 fit() 메서드를 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 도와준다.

In [62]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True, random_state=42)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break  # early stopping

In [63]:
print(gbrt.n_estimators)

73


In [64]:
print("Minimum validation MSE:", min_val_error)

Minimum validation MSE: 0.0030248830463853182


- 연속해서 다섯 번의 반복 동안 검증 오차가 향상되지 않으면 훈련을 멈춘다.
- 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정할 수 있는 subsample 매개변수도 지원한다.
> - 예를들어 subsample = 0.25라고 하면 각 트리는 무작위로 선택된 25%의 훈련 샘플로 학습된다.
> - 편향이 높아지는 대신 분산이 낮아지게 될 것이다. 또한 훈련 속도를 상당히 높인다. 이러한 기법을 확률적 그래디언트 부스팅이라 한다.

In [65]:
try:
    import xgboost
except ImportError as ex:
    print("Error: the xgboost library is not installed.")
    xgboost = None

In [66]:
if xgboost is not None:  # not shown in the book
    xgb_reg = xgboost.XGBRegressor(random_state=42)
    xgb_reg.fit(X_train, y_train)
    y_pred = xgb_reg.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    print("Validation MSE:", val_error)

Validation MSE: 0.0031085575071820525


In [67]:
if xgboost is not None:  # not shown in the book
    xgb_reg.fit(X_train, y_train,
                eval_set=[(X_val, y_val)], early_stopping_rounds=2)
    y_pred = xgb_reg.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    print("Validation MSE:", val_error)

[0]	validation_0-rmse:0.270336
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.246844
[2]	validation_0-rmse:0.22505
[3]	validation_0-rmse:0.205587
[4]	validation_0-rmse:0.188344
[5]	validation_0-rmse:0.172722
[6]	validation_0-rmse:0.158756
[7]	validation_0-rmse:0.146858
[8]	validation_0-rmse:0.13578
[9]	validation_0-rmse:0.125776
[10]	validation_0-rmse:0.116912
[11]	validation_0-rmse:0.109149
[12]	validation_0-rmse:0.102133
[13]	validation_0-rmse:0.095785
[14]	validation_0-rmse:0.090162
[15]	validation_0-rmse:0.085444
[16]	validation_0-rmse:0.081195
[17]	validation_0-rmse:0.077256
[18]	validation_0-rmse:0.073665
[19]	validation_0-rmse:0.071101
[20]	validation_0-rmse:0.068513
[21]	validation_0-rmse:0.066321
[22]	validation_0-rmse:0.064664
[23]	validation_0-rmse:0.063302
[24]	validation_0-rmse:0.062233
[25]	validation_0-rmse:0.061267
[26]	validation_0-rmse:0.060281
[27]	validation_0-rmse:0.059617
[28]	validation_0-rmse:0.058976
[29]	validation_0-rm

## 스태킹
- 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련시킬 수 없을까? 라는 기본적인 아이디어에서 출발한다.
- 스태킹(stacking, stacked generalization의 줄임)은 '1 - 투표 기반 분류기'에서 처럼 'hard voting', 'soft voting' 방법이 아니라 앙상블 학습에서 각 모델의 예측값을 가지고 새로운 메타 모델(meta learner)을 학습시켜 최종 예측 모델을 만드는 방법을 말한다.

![test](./img/스태킹.png)

- 블렌더를 학습시키는 일반적인 방법은 홀드 아웃 세트를 사용하는 것이다. 
- 먼저 훈련 세트를 두 개의 서브셋으로 나누고, 첫 번째 서브셋은 첫번째 레이어의 예측을 훈련 시키기 위해 사용된다.
- 그런 다음 첫 번째 레이어의 예측기를 사용해 두 번째 세트 홀드 아웃 세트에 대한 예측을 만든다. 예측기들이 훈련하는 동안 이 샘플들을 전혀 보지 못했기 때문에 이때 만들어진 예측은 완전히 새로운 것이다. 
- 이제 홀드 아웃 세트의 각 샘플에 대해 세 개의 예측값이 만들어진다.
- 타깃값은 그대로 쓰고 앞에서 예측한 값을 입력 특성으로 사용하는 사용하는 새로운 훈련 세트를 만들 수 있다. (새로운 훈련 세트는 3차원이 된다.) 블렌더가 새 훈련 세트로 훈련이 된다. 즉, 첫번째 레이어의 예측을 가지고 타깃값을 예측하도록 학습된다.

스태킹(stacking)의 과정은 다음과 같다.

- 학습 데이터셋에서 샘플링을 통해 서브셋1(subset-1)을 만들고, 이 서브셋을 이용해 각 모델을 학습시킨다. 

- 서브셋2(subset-2) 학습 시킨 모델을 이용해 각 모델의 예측값을 출력하고 예측값들을 합친다.

- 합쳐진 예측값들을 입력 특성(input feature)로 사용하는 새로운 모델(meta learner, blender)을 학습시킨다.



